### Следить за последним файлом
Если он не обнавляется более 1 часа, то писать в телеграм предупреждение о возможно ошибке в работе прибора.

In [63]:
import os, time
import struct
import pandas as pd

#specify that all columns should be shown
pd.set_option('max_columns', None)


##  найти самый поздний файл
def get_latest_file(dirname):
    ''' 
    Функция ищет самый поздний файл
    Возвращает его имя
    '''
    max_file = 'Initial string'
    sep = '/'
    if not dirname.endswith('/'):
        dirname = dirname + sep
    if not os.path.isdir(dirname):
        print(f"Alarm!! Нет такой папки {dirname}! Валим отсюда!")
        return "Error" ## \todo вернуть сообщение об ошибке

    max_atime = 0
    print(os.listdir(dirname))
    for filename in os.listdir(dirname):
        ## проверить файл ли это
        if not os.path.isfile(dirname + filename):
            continue
        if filename[-3:] != 'O30':
            continue
        if os.path.getctime(dirname + filename) > max_atime:
            max_atime = os.path.getctime(dirname + filename)
            max_file = dirname + filename
    return max_file



#############################################
path = '../work/data'

## открыть самый последний файл
if not os.path.isdir(path):
    print(f"Alarm!! Нет такой папки {path}! Валим отсюда!")
    #return "Error" ## \todo вернуть сообщение об ошибке
else:
    current_file = get_latest_file(path)
    print(current_file)
    if not current_file.endswith('O30'):
        print(f"Alarm! No data file in the directory {path}")
    else:    
        now = time.time() ## текущее время
        delta = (now - os.path.getctime(current_file)) // 60 ## minutes
        if delta > 60: # 60 min
            print(f"Alarm!!! File is {delta / 60:.0f} hours old!")
            ## \todo послать предупреждение, что самый поздний файл очень старый
        print(f"Файл {current_file} изменен {delta / 60:.0f} часов ({delta:.0f} минут) назад")


['2021-09-28 05 numb row.txt', '20+5 мин.53.O30', '20+5 мин.54.O30', 'Satino2022_1.26.O30', '20+5 мин RAW.txt', 'Satino2022_1.16.O30', '2021-09-28 04 mass log row.txt', '2021-09-28 03 numb log row.txt']
../work/data/20+5 мин.54.O30
Файл ../work/data/20+5 мин.54.O30 изменен 1 часов (48 минут) назад


### Конвертер из бинарного в текстовый
Открыть, прочитать, записать исходные данные в текстовый файл.

In [116]:
unknown_text_in_header = """Alarm Set,No
Dead Time Correction Applied,Yes
Units,dW/dlogDp
Weight,Number
Refractive Index Applied,No
"""


def read_floats(data, n, size):
    return struct.unpack('d'*size, data[n:n + 8 * size])


def translate_header(header):
    '''
    расшифровать заголовок и вернуть границы интервалов
    '''
    model_byte = 64
    sernum_byte = model_byte + 20
    model = header[model_byte:model_byte + 4].decode()
    sernum = header[sernum_byte:sernum_byte + 10].decode()
    sample_length_byte = 2170 * 4
    sample_length = struct.unpack('i', header[sample_length_byte:sample_length_byte + 4])[0]
    boundaries = read_floats(header, 8692, 17) ## Boundaries
 
    print("Sample File," + current_file)
    print(f"Instrument Model,{model}")
    print(f'Instrument Serial Number,{sernum}')
    print(f'Sample Length (s),{sample_length}')
    print(unknown_text_in_header)
    print(',,,,,,,,,,,,,,,,LB,' + ','.join(map(str,boundaries[:-1])))
    print(',,,,,,,,,,,,,,,,UB,' + ','.join(map(str,boundaries[1:])))
    print(',,,,,,,,,,,,,,,,LB with RI,' + ','.join(map(str,boundaries[:-1])) + ',')
    print(',,,,,,,,,,,,,,,,UB with RI,' + ','.join(map(str,boundaries[1:])) + ',')

    return boundaries


## читать из данных события значения в каналах
def read_channels(data):
    chan_len = 20 ## длина записи данных одного канала в байтах
    channels = []
    for i in range(15):
        ch = struct.unpack('dhhii', data[i * chan_len: (i + 1)* chan_len])
        #channels.append(ch)
        channels += ch
    return channels


def generate_csv_header():
    text_header = ['lenght', 'datatime', 'date', 'time', 'x1', 'x2', 'x3'] + \
                  ['y'+str(i) for i in range(1, 21)] + ['x4', 'temperature', 'pressure', 'humidity']
    chan_header = [x + str(i)  for i in range(1,16) for x in ["dt" , 'tr', 'al', 'vl', 'nch'] ]
    text_header += chan_header
    text_header += ['m' + str(i) for i in range(1, 4)] # + ['TotalConc']
    #text_header += ['z' + str(i) for i in range(0, 16)] + ['end']
    text_header += [f"{(LB[i] + UB[i]) * 0.5:.3f}" for i in range(0, 16)] + ['other']
    ## tr14 - Errors:
    text_header[text_header.index('tr14')] = "Errors"
    ## dt14 - Dead Time - заменить в заголовке
    text_header[text_header.index('dt14')] = "Dead Time"
    return text_header


def read_one_event(file_with_data):
    '''  Читаем одну запись  '''
    line_length = 658 ## длина одной строки данных
    chan_pos = 220    ## начало записи данных каналов в байтах
    chan_len = 300    ## длина записи данных всех каналов
    last_pos = 534 - 8    ## начало последних данных в записи, позиция первого байта после канало

    ## read binary data from file
    data_byte = file_with_data.read(line_length)
    
    ## check length of read line
    if len(data_byte) < line_length:
        print(f"\nError in data!!!! len = {len(data_byte)} bytes but expected {line_length} bytes" )
        return

    ## check first two chars: if no 92 02 - no data
    if not data_byte.startswith(b'\x92' + b'\x02'):
        print(f"\nError in data!! No standart start bytes!!! No standart length of event data") ## \todo правильно обработать эту ошибку
        return
    
    ### read first bytes
    format_first = 'ii6hiiI' + 23 * 'd' +'f'
    firstdata = list(struct.unpack(format_first, data_byte[:chan_pos]))
    ## соберем дату
    firstdata.insert(2, str(firstdata[4]) + '-' + str(f'{firstdata[2]:02d}') + '-' + str(f'{firstdata[3]:02d}'))
    [firstdata.pop(3) for _ in range(3)]
    ## соберем время
    firstdata.insert(3, str(f'{firstdata[3]:02d}') + ':' + str(f'{firstdata[4]:02d}') + ':' + str(f'{firstdata[5]:02d}'))
    [firstdata.pop(4) for _ in range(3)]

    ### read channels 
    channels = read_channels(data_byte[chan_pos:])

    ### read 7 integers after channels
    #print(chan_pos + chan_len + 5 * 2, last_pos)
    middle  = list(struct.unpack('3H', data_byte[chan_pos + chan_len: last_pos]))
    #middle += list(struct.unpack('f',  data_byte[chan_pos + chan_len + 5 * 2: last_pos]))

    ### read last bytes
    last_num = struct.unpack('16dI', data_byte[last_pos:])

    alldata = list(firstdata) + list(channels) + list(middle) + list(last_num)
    return alldata


##############################################

##############################################

current_file = 'data/Satino2022_1.26.O30'
current_file = 'data/20+5 мин.54.O30'
## читать бинарный файл
file_with_data = open(current_file, "rb")

## Читаем заголовок из файла
header_length = 9172
header_data = file_with_data.read(header_length)
boundaries = translate_header(header_data)
LB = boundaries[:-1]
UB = boundaries[1:]
dB = [(u - l) for l, u in zip(LB, UB)] ## dDp

### make dataframe
text_header = generate_csv_header()
df = pd.DataFrame(columns=text_header)

## Читаем файл, пока считываются строки по 658 символов \todo заменить число на переменную
event_length = 658
n = 0
reading = True
while True:
    ## read one event
    alldata = read_one_event(file_with_data)
    if not alldata:
        break

    ### make dictionary
    newline = {x:y for x,y in zip(text_header, alldata)}
    df = pd.concat([df,pd.DataFrame([newline])], ignore_index=True)

#print(firstdata, channels, middle, last_num, sep='\n-----------------\n')
#print(alldata)
#print(last_num)
#df['dt14']
df.to_csv(current_file + ".txt", sep=' ') 
df.to_csv(current_file + ".csv")    

## tr14 - Errors:
## 48= Flow Error;Flow Blocked
## 0 = No Errors
## 264 = System Error;Flow Blocked Instrument Stopped

## dt14 - Dead Time - заменить в заголовке

Sample File,data/20+5 мин.54.O30
Instrument Model,3330
Instrument Serial Number,3330200403
Sample Length (s),1200
Alarm Set,No
Dead Time Correction Applied,Yes
Units,dW/dlogDp
Weight,Number
Refractive Index Applied,No

,,,,,,,,,,,,,,,,LB,0.3,0.44,0.56,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.533,2.95,3.463,5.61,8.032
,,,,,,,,,,,,,,,,UB,0.44,0.56,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.533,2.95,3.463,5.61,8.032,10.0
,,,,,,,,,,,,,,,,LB with RI,0.3,0.44,0.56,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.533,2.95,3.463,5.61,8.032,
,,,,,,,,,,,,,,,,UB with RI,0.44,0.56,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.533,2.95,3.463,5.61,8.032,10.0,

Error in data!!!! len = 0 bytes but expected 658 bytes


In [117]:
data = pd.read_csv(current_file + ".csv", index_col=0)
data

,lenght,datatime,date,time,x1,x2,x3,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,x4,temperature,pressure,humidity,dt1,tr1,al1,vl1,nch1,dt2,tr2,al2,vl2,nch2,dt3,tr3,al3,vl3,nch3,dt4,tr4,al4,vl4,nch4,dt5,tr5,al5,vl5,nch5,dt6,tr6,al6,vl6,nch6,dt7,tr7,al7,vl7,nch7,dt8,tr8,al8,vl8,nch8,dt9,tr9,al9,vl9,nch9,dt10,tr10,al10,vl10,nch10,dt11,tr11,al11,vl11,nch11,dt12,tr12,al12,vl12,nch12,dt13,tr13,al13,vl13,nch13,Dead Time,Errors,al14,vl14,nch14,dt15,tr15,al15,vl15,nch15,m1,m2,m3,0.370,0.500,0.680,0.900,1.100,1.300,1.500,1.700,1.900,2.100,2.367,2.742,3.207,4.537,6.821,9.016,other
0,658,1632776436,2021-09-28,00:00:36,0,0,68392,1.062225e+158,0.0,2.121996e-314,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,31.319,100.259,0.0,0.0,9436,1,1,1623129640,0.0,10094,1,1,1623129940,0.0,10752,1,1,1623130240,0.0,11410,1,1,1623130540,0.0,12068,1,1,1623130840,0.0,12726,1,1,1623131140,0.0,13384,1,1,1623131440,0.0,14042,1,1,1623131740,0.0,14700,1,1,1623132040,0.0,15358,1,1,1623132340,0.0,16016,1,1,1623132640,0.0,16674,1,1,1623132940,0.0,17332,1,1,1623133240,2.624839,0,0,0,1623133540,0.0,18648,1,1,1623133840,16,0,16,408368.0,33316.0,11189.0,3530.0,1170.0,921.0,719.0,788.0,683.0,491.0,514.0,432.0,346.0,552.0,127.0,13.0,0
1,658,1632777637,2021-09-28,00:20:37,0,0,68392,1.062225e+158,0.0,2.121996e-314,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,31.349,100.290,0.0,0.0,9436,1,1,1623129640,0.0,10094,1,1,1623129940,0.0,10752,1,1,1623130240,0.0,11410,1,1,1623130540,0.0,12068,1,1,1623130840,0.0,12726,1,1,1623131140,0.0,13384,1,1,1623131440,0.0,14042,1,1,1623131740,0.0,14700,1,1,1623132040,0.0,15358,1,1,1623132340,0.0,16016,1,1,1623132640,0.0,16674,1,1,1623132940,0.0,17332,1,1,1623133240,2.934016,0,0,0,1623133540,0.0,18648,1,1,1623133840,16,0,16,454993.0,37432.0,12852.0,4020.0,1362.0,1014.0,779.0,904.0,759.0,497.0,687.0,504.0,390.0,631.0,146.0,21.0,0
2,658,1632778837,2021-09-28,00:40:37,0,0,68392,1.062225e+158,0.0,2.121996e-314,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,31.319,100.290,0.0,0.0,9436,1,1,1623129640,0.0,10094,1,1,1623129940,0.0,10752,1,1,1623130240,0.0,11410,1,1,1623130540,0.0,12068,1,1,1623130840,0.0,12726,1,1,1623131140,0.0,13384,1,1,1623131440,0.0,14042,1,1,1623131740,0.0,14700,1,1,1623132040,0.0,15358,1,1,1623132340,0.0,16016,1,1,1623132640,0.0,16674,1,1,1623132940,0.0,17332,1,1,1623133240,2.977107,0,0,0,1623133540,0.0,18648,1,1,1623133840,16,0,16,460341.0,38165.0,13013.0,3961.0,1280.0,1111.0,787.0,822.0,747.0,521.0,659.0,540.0,457.0,641.0,150.0,27.0,0
3,658,1632780037,2021-09-28,01:00:37,0,0,68392,1.062225e+158,0.0,2.121996e-314,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,31.319,100.290,0.0,0.0,9436,1,1,1623129640,0.0,10094,1,1,1623129940,0.0,10752,1,1,1623130240,0.0,11410,1,1,1623130540,0.0,12068,1,1,1623130840,0.0,12726,1,1,1623131140,0.0,13384,1,1,1623131440,0.0,14042,1,1,1623131740,0.0,14700,1,1,1623132040,0.0,15358,1,1,1623132340,0.0,16016,1,1,1623132640,0.0,16674,1,1,1623132940,0.0,17332,1,1,1623133240,2.797012,0,0,0,1623133540,0.0,18648,1,1,1623133840,16,0,16,431152.0,36221.0,12339.0,3775.0,1212.0,956.0,739.0,775.0,743.0,498.0,643.0,527.0,399.0,683.0,126.0,16.0,0
4,658,1632781237,2021-09-28,01:20:37,0,0,68392,1.062225e+158,0.0,2.121996e-314,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,31.319,100.322,0.0,0.0,9436,1,1,1623129640,0.0,10094,1,1,1623129940,0.0,10752,1,1,1623130240,0.0,11410,1,1,1623130540,0.0,12068,1,1,1623130840,0.0,12726,1,1,1623131140,0.0,13384,1,1,1623131440,0.0,14042,1,1,1623131740,0.0,14700,1,1,1623132040,0.0,15358,1,1,1623132340,0.0,16016,1,1,1623132640,0.0,16674,1,1,1623132940,0.0,17332,1,1,1623133240,2.698148,0,0,0,1623133540,0.0,18648,1,1,1623133840,16,0,16,415518.0,34543.0,11818.0,3617.0,1196.0,982.0,715.0,826.0,728.0,568.0,677.0,515.0,426.0,685.0,144.0,19.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

### Конвертер из текстового файла в математику

In [84]:
## tr14 - ошибки:
## 48= Flow Error;Flow Blocked
## 0 = No Errors
## 264 = System Error;Flow Blocked Instrument Stopped
print(f"{bin(48):10s}", "Flow Error;Flow Blocked")
print(bin(264), "Flow Error;Flow Blocked Instrument Stopped")
##    110000   Flow Error;Flow Blocked
## 100001000   Flow Error;Flow Blocked Instrument Stopped

0b110000   Flow Error;Flow Blocked
0b100001000 Flow Error;Flow Blocked Instrument Stopped


## ========================================================

In [56]:
## прочитать данные до признака начала строки данных 92 02
#def read_header(filename)

current_file = 'data/Satino2022_1.16.O30'
#current_file = 'data/20+5 мин.53.O30'
file_with_data = open(current_file, "rb")
data = file_with_data.read()
print(len(data), data)
data.find(b'\x92' + b'\x02')

20358 b';OPS (C) TSI Inc 2010-566B5E90-CDEA-11CF-9AA0-0000C9251D64\r\n\x00\x01\x00\x003330\x00+\x17|<,\x17|\x00\x00\x00\x00\x00\x00\x00\x003330200403\x00\x00\x00\x00\x00\x00\x00\x00\x00\x004G\x16|\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00Satino2022_1.15.O30\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

9172

In [30]:
b'\x92' + b'\x02'

b'\x92\x02'

In [28]:
x = (chr(0x92)+chr(0x02)).encode()
x

b'\xc2\x92\x02'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4888ebe5-2b59-4c64-9265-248a80232ba5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>